In [1]:
import pandas as pd
import numpy as np
import warnings
import math
import os
import lightgbm as lgb
from  sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import roc_auc_score,f1_score,cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [2]:
def model_train(df,trainlabel,cate_cols,test_,feature,num_class):
    '''
    @param df: 训练数据 DataFrame
    @param trainlabel：训练标签 string  eg. 'label'
    @param cate_cols: 类别变量名 list  eg. ['col1','col2'...]
    @param test_ : 测试数据 DataFrame
    @param feature ：所有训练特征 list  eg. ['feat1','feat2'...]

    @return sub_preds: 预测数据
    
    '''
    train_= df.copy()
    auc = []
    n_splits = 5
    oof_lgb = np.zeros([len(train_),num_class])
    folds = KFold(n_splits=n_splits, shuffle=True, random_state=2019)
    stratifiedKfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2019)
    sub_preds = np.zeros([test_.shape[0],num_class])
    sub_preds1 = np.zeros([test_.shape[0],n_splits])
    use_cart = True
    cate_cols = cate_cols
    label = trainlabel
    pred = list(feature)
    params = {
        'learning_rate': 0.02,
        'boosting_type': 'gbdt',
        'objective':'multiclass',
        #'metric':'multi-error',
        'num_class':num_class,
        'num_leaves':60,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'seed': 1,
        'bagging_seed': 1,
        'feature_fraction_seed': 5,
        'min_data_in_leaf': 20,
        'max_depth':-1,
        'nthread': 8,
        'verbose': 1,
 #       'is_unbalanace':True,
#         'lambda_l1': 0.4,  
#         'lambda_l2': 0.5, 
        'device': 'cpu'
    }
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_[pred],train_[[label]]), start=1):
        print('the %s training start ...'%n_fold)

        train_x, train_y,train_weight = train_[pred].iloc[train_idx], train_[[label]].iloc[train_idx],train_[['weight']].iloc[train_idx]
        valid_x, valid_y,valid_weight = train_[pred].iloc[valid_idx], train_[[label]].iloc[valid_idx],train_[['weight']].iloc[valid_idx]
        #x = train_['bid'].iloc[valid_idx]
        print(train_y.shape)
        if use_cart:
            dtrain = lgb.Dataset(train_x, label=train_y, categorical_feature=cate_cols,weight=train_weight.values.flatten(order='F'))
            dvalid = lgb.Dataset(valid_x, label=valid_y, categorical_feature=cate_cols)
            #dvalid1 = lgb.Dataset(test_[pred], label=test_[['label']], categorical_feature=cate_cols)

        else:
            dtrain = lgb.Dataset(train_x, label= train_y)
            dvalid = lgb.Dataset(valid_x, label= valid_y)

        clf = lgb.train(
            params=params,
            train_set=dtrain,
            num_boost_round=2000,
            valid_sets=[dvalid],
            early_stopping_rounds = 100,
            verbose_eval=100
           ,feval=eval_error
        )
        
        sub_preds += clf.predict(test_[pred].values,num_iteration=1000)/ folds.n_splits
        sub_preds1[:,n_fold-1] = clf.predict(test_[pred].values,num_iteration=400).argmax(axis=1)
        train_pred = clf.predict(valid_x,num_iteration=clf.best_iteration)
        y_pred = train_pred.argmax(axis=1)
        oof_lgb[valid_idx] = train_pred
    #print('MEAN AUC:',np.mean(auc))
    
    return sub_preds,oof_lgb,clf,sub_preds1

In [6]:
cust_avli_Q3 = pd.read_csv(r'D:\学习\厦门银行/train/avli/cust_avli_Q3.csv')
cust_avli_Q4 = pd.read_csv(r'D:\学习\厦门银行/train/avli/cust_avli_Q4.csv')
cust_avli_test = pd.read_csv(r'D:\学习\厦门银行/test/avli/cust_avli_Q1.csv')
label_Q3 = pd.read_csv(r'D:\学习\厦门银行/train/y/y_Q3_3.csv')
label_Q4 = pd.read_csv(r'D:\学习\厦门银行/train/y/y_Q4_3.csv')
label_Q3['label'] = label_Q3.label + 1
label_Q4['label'] = label_Q4.label + 1
cust_avli_Q3 = cust_avli_Q3.merge(label_Q3,on='cust_no',how='left')
cust_avli_Q4 = cust_avli_Q4.merge(label_Q4,on='cust_no',how='left')

cust_avli_Q3['pre_label'] = np.nan
last_label = label_Q3[['cust_no','label']]
last_label.columns = ['cust_no','pre_label']
cust_avli_Q4 = cust_avli_Q4.merge(last_label,on='cust_no',how='left')

last_label = label_Q4[['cust_no','label']]
last_label.columns = ['cust_no','pre_label']
cust_avli_test = cust_avli_test.merge(last_label,on='cust_no',how='left')

In [9]:
Train_data= pd.read_csv(r'D:\学习\厦门银行/1210/train.csv')
feature = Train_data.drop(['cust_no','label','weight'],axis=1).columns
cust_avli_test= pd.read_csv(r'D:\学习\厦门银行/1210/cust_avli_test.csv')

In [ ]:
def eval_error(pred,train_set):
    
    labels = train_set.get_label()
    pred = pred.reshape((3,int(len(pred)/3))).T
    y_pred = pred.argmax(axis=1)
    score = cohen_kappa_score(labels,y_pred)
    return 'kappa_score',score,True

sub_preds,oof_lgb,clf,sub_preds1 = model_train(Train_data,trainlabel='label',cate_cols=[],test_=cust_avli_test,feature=feature,num_class=3)

the 1 training start ...
(116236, 1)
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's multi_logloss: 0.682143	valid_0's kappa_score: 0.457145
[200]	valid_0's multi_logloss: 0.647355	valid_0's kappa_score: 0.467509
[300]	valid_0's multi_logloss: 0.638277	valid_0's kappa_score: 0.472822
[400]	valid_0's multi_logloss: 0.634641	valid_0's kappa_score: 0.475312
[500]	valid_0's multi_logloss: 0.63223	valid_0's kappa_score: 0.47774
[600]	valid_0's multi_logloss: 0.630628	valid_0's kappa_score: 0.479582
[700]	valid_0's multi_logloss: 0.629324	valid_0's kappa_score: 0.480184
Early stopping, best iteration is:
[645]	valid_0's multi_logloss: 0.630061	valid_0's kappa_score: 0.480615
the 2 training start ...
(116237, 1)
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's multi_logloss: 0.686107	valid_0's kappa_score: 0.450416
[200]	valid_0's multi_logloss: 0.652205	valid_0's kappa_score: 0.46286
[300]	valid_0's multi_logloss: 0.643542	valid_0'

In [22]:
cust_avli_test['label'] = sub_preds.argmax(axis=1) - 1
cust_avli_test[['cust_no','label']].to_csv(r'D:\学习\厦门银行/1210/baseline3.csv',index=None)